## LeNet

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential
import matplotlib.pyplot as plt
import random
import numpy as np

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# get shape of x_train
x_train.shape

In [ ]:
# Reshape and normalize data
x_train = x_train.reshape(60000, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32') / 255.0
# reshape y_train and y_test
y_train = y_train.reshape(60000, 1)
y_test = y_test.reshape(10000, 1)

In [ ]:
# Create a simple model
model = Sequential([
    Input(shape = (28, 28, 1)),
    Conv2D(filters=6, kernel_size=5, activation='sigmoid', padding='same'),
    AvgPool2D(pool_size=2, strides=2),
    Conv2D(filters=16, kernel_size=5,activation='sigmoid'),
    AvgPool2D(pool_size=2, strides=2),
    Flatten(),
    Dense(120, activation='sigmoid'),
    Dense(84, activation='sigmoid'),
    Dense(10, activation='softmax')
])


In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=25, batch_size=512, validation_split=0.1)

In [ ]:
# test random input

i = random.randint(0,x_test.shape[0])
print(np.argmax(model.predict(x_test[i:i+1])), y_test[i].T)
plt.imshow(x_test[i]);

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_prob = model.predict(x_test)
y_pred = np.argmax(y_prob, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
plt.plot(history.history['loss']);
plt.plot(history.history['val_loss']);

## AlexNet

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential
import matplotlib.pyplot as plt
import random
import numpy as np
from keras.datasets import mnist
from numba import cuda
from tqdm import tqdm

2025-02-26 20:02:13.887310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740618133.992475  535269 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740618134.022571  535269 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = mnist.load_data()

In [3]:
# Reshape and normalize data
x_train_np = x_train_raw[::5].reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test_np = x_test_raw[::5].reshape(-1, 28, 28, 1).astype('float32') / 255.0
# reshape y_train and y_test
y_train = y_train_raw.reshape(-1, 1)
y_test = y_test_raw.reshape(-1, 1)

In [4]:
def resize_images(images, new_size, batch_size=500):
    resized_images = []
    for i in tqdm(range(0, len(images), batch_size)):
        batch = images[i:i+batch_size]
        with tf.device('/GPU:0'):
            resized_batch = tf.image.resize(batch, new_size)
        resized_images.append(resized_batch)
    with tf.device('/GPU:0'):
        return tf.concat(resized_images, axis=0)

In [5]:
x_train = resize_images(x_train_np[::3], [224, 224])
x_test = resize_images(x_test_np[::3], [224, 224])
x_train.shape

100%|██████████| 2/2 [00:00<00:00, 87.44it/s]


TensorShape([4000, 224, 224, 1])

In [6]:
model = Sequential([
    Input(shape = (224, 224, 1)),
    Conv2D(filters=96, kernel_size=11, strides=4,activation='relu'),
    MaxPool2D(pool_size=3, strides=2),
    Conv2D(filters=256, kernel_size=5, padding='same',activation='relu'),
    MaxPool2D(pool_size=3, strides=2),
    Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
    Conv2D(filters=384, kernel_size=3, padding='same',activation='relu'),
    Conv2D(filters=256, kernel_size=3, padding='same',activation='relu'),
    MaxPool2D(pool_size=3, strides=2),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 54, 54, 96)     │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 26, 26, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    26,218,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        40,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,764,746 (178.39 MB)

 Trainable params: 46,764,746 (178.39 MB)

 Non-trainable params: 0 (0.00 B)